# Quantum Circuits
Quantum computers can only use a specific set of gates (universal gate set). Given the entanglers and their amplitudes found in Step 3, one can find corresponding representation of these operators in terms of elementary gates using the following procedure.

In [10]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']= 'True'

import tequila as tq
from utility import *

First, we set up the Hamiltonian in Tequila's format and the unitary gates obtained in Step 3. 

In [12]:
H = tq.QubitHamiltonian.from_openfermion(get_qubit_hamiltonian('h2', 2, 'sto-3g', qubit_transf='jw'))

a = tq.Variable("tau_0")
U = construct_QMF_ansatz(4)
U += tq.gates.ExpPauli(paulistring=tq.PauliString.from_string("X(0)Y(1)X(2)X(3)"), angle=a)
print(U)

circuit: 
Rx(target=(0,), parameter=beta_0)
Rz(target=(0,), parameter=gamma_0)
Rx(target=(1,), parameter=beta_1)
Rz(target=(1,), parameter=gamma_1)
Rx(target=(2,), parameter=beta_2)
Rz(target=(2,), parameter=gamma_2)
Rx(target=(3,), parameter=beta_3)
Rz(target=(3,), parameter=gamma_3)
Exp-Pauli(target=(0, 1, 2, 3), control=(), parameter=tau_0, paulistring=X(0)Y(1)X(2)X(3))



One can check the expectation value to see it is near the ground state energy.

In [3]:
E = tq.ExpectationValue(H=H, U=U)
vars = {'beta_1': 3.141592624143881, 'beta_0': 3.141592624143881, 'tau_0': 1.1331410014096885, 'gamma_1': 0.0, 'beta_3': 0.0, 'gamma_3': 0.0, 'gamma_2': 0.0, 'gamma_0': 0.0, 'beta_2': 0.0} # values obtained from step 3
print(tq.simulate(E, variables=vars))

-0.9486411121761621


One can run the same experiment on a real quantum computer through IBM Quantum Experience (ibmq). After activating your account here (https://quantum-computing.ibm.com/login), copy the API token and execute the commented block below. 

In [4]:
# from qiskit import IBMQ
# IBMQ.save_account('MY_API_TOKEN')

In [5]:
# list of devices available can be found in ibmq account page
# Try a device that isn't too busy. If the wait time is too long, measure only one pauli-word in the H2 Hamilotonian using the commented line
#E = tq.ExpectationValue(H=tq.QubitHamiltonian.from_paulistrings("X(0)X(1)Y(2)Y(3)"), U=U)
tq.simulate(E, variables=vars, samples=100, backend="qiskit", device='ibmq_manila')

-0.7276849297174877

The following code block prints the circuit.

In [6]:
circ = tq.circuit.compiler.compile_exponential_pauli_gate(U)
tq.draw(circ, backend="qiskit")

┌────────────────────┐┌─────────────────────┐   ┌───┐                   »
q_0: ┤ RX(f((beta_0,))_0) ├┤ RZ(f((gamma_0,))_1) ├───┤ H ├──────■────────────»
     ├────────────────────┤├─────────────────────┤┌──┴───┴───┐┌─┴─┐          »
q_1: ┤ RX(f((beta_1,))_2) ├┤ RZ(f((gamma_1,))_3) ├┤ RX(pi/2) ├┤ X ├──■───────»
     ├────────────────────┤├─────────────────────┤└──┬───┬───┘└───┘┌─┴─┐     »
q_2: ┤ RX(f((beta_2,))_4) ├┤ RZ(f((gamma_2,))_5) ├───┤ H ├─────────┤ X ├──■──»
     ├────────────────────┤├─────────────────────┤   ├───┤         └───┘┌─┴─┐»
q_3: ┤ RX(f((beta_3,))_6) ├┤ RZ(f((gamma_3,))_7) ├───┤ H ├──────────────┤ X ├»
     └────────────────────┘└─────────────────────┘   └───┘              └───┘»
c_0: ════════════════════════════════════════════════════════════════════════»
                                                                             »
c_1: ════════════════════════════════════════════════════════════════════════»
                                                         